<div style="color:black; background-color:#FFF3E9; border: 1px solid #FFE0C3; border-radius: 10px; margin-bottom:0rem">
    <p style="margin:1rem; padding-left: 1rem; line-height: 2.5;">
        ©️ <b><i>Copyright 2024 @ Author</i></b><br/>
        <i>Author：
            <b>
            <a href="mailto:your_address@email.com">Wang Jianghai 📨 </a>
            </b>
        </i>
        <br/>
        <i>Date：2024-03-19</i><br/>
        <i>License：</a><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">CC BY-NC-SA 4.0 International</a></i><br/>
</div>

> **Code Implementation**：
> 
> - [Python Implementation: ***Lennard-Jones Potential***](https://nb.bohrium.dp.tech/detail/35387325971)
> - [Python Implementation: ***Optimization Algorithms for Molecular Dynamics***](https://nb.bohrium.dp.tech/detail/36773325971)
> - [Python Implementation: ***Integration Algorithms for Molecular Dynamics***](https://nb.bohrium.dp.tech/detail/49396512168)

# Introduction to Computational Materials Science

---


## 1. Classification of computation methods

### 1.1 Nanoscale

**Elementary Particle**: Electron

**Main Theory**: Quantum Mechanics

$$
i\hbar\frac{\partial\psi}{\partial t}=-\frac{\hbar^2}{2m}\frac{\partial^2\psi}{\partial x^2}
$$

**Main Method**: First-principles calculations based on quantum chemistry and density functional theory (DFT)

**Common Software**: VASP, Gaussian, QuantumEspresso

**Calculation Content**:

- Structural Properties: Molecular and crystal structure prediction

- Surface Properties: Reconstruction, defects, surface energy

- Mechanical Properties: Elastic constants, Young's modulus

- Magnetic Properties: Magnetism, spin-orbit coupling

- Electronic Properties: Molecular orbitals, band structure, valence states

- Optical Properties: Absorption spectrum, refractive index

- Dynamic Simulations: Diffusion coefficient, reaction kinetics

### 1.2 Microscopic scale

> When the atomic [**de Broglie wavelength**](https://en.wikipedia.org/wiki/Matter_wave "de Broglie waves, also called matter waves, are a central part of quantum mechanics and an example of wave-particle duality. The theory states that all matter behaves like waves.") is much smaller than the lattice constant, the atomic motion can be approximately described by Newtonian classical mechanics.

**Elementary particles**: Atoms

**Main theory**: Newtonian mechanics

$$
\overrightarrow{F}=m\overrightarrow{a}
$$

**Main Methods**: Monte Carlo (MC), Molecular Dynamics (MD)

**Common Software**: LAMMPS, GROMACS

**Calculation Content**:

- Mechanical Properties: Microscopic Plastic Deformation Mechanisms, Tensile Fracture Mechanisms

- Thermal Properties: Phase Transitions, Thermal Expansion Coefficient

- Material Structure/Interactions: Adsorption, Diffusion, Defect Motion, Supramolecular Self-Assembly, Surface Energy

### 1.3 Mesoscopic scale

**Elementary particles**: Coarse particles (or tissues, structures)

**Main theories**: Newtonian mechanics, statistical thermodynamics

**Main methods**:

- Lattice Boltzmann (LBM)

- Dissipative particle dynamics (DPD)

- Brownian dynamics (BD)

- Dislocation dynamics (DDD)

- Phase transition dynamics (PTD)

**Common software**: LAMMPS, GROMACS

### 1.4 Macroscale

**Elementary particles**: Continuum

**Main theories**: Theoretical mechanics, fluid mechanics

**Main methods**:

- Finite element method (FEM)

- Finite difference method (FDM)

- Finite volume method (FVM)

Common software: ANSYS, ABAQUS

**Calculation content**:

- Impact damage simulation

- Force/thermal/electrical/magnetic field coupling analysis

- Temperature distribution simulation

- Cutting surface formation process

- Crack growth and stress distribution, etc.



## 2. Basic Computation Process for Materials

1. Determine the Physical Model (Sources of Error)

2. Select the Numerical Method (Sources of Error)

3. Analyze the Calculation Results (Selection of Numerical Results)

4. Derive the Physical Conclusion

# Introduction to Molecular Dynamics

---


> [Molecular dynamics](https://en.wikipedia.org/wiki/Molecular_dynamics)（MD）is a computational technique that uses **numerical simulations** to study the time evolution of molecular, atomic, or particle systems under **classical mechanics**. MD allows for the observation and analysis of the dynamic behavior of molecular systems at the microscopic scale, establishing relationships between the microscopic and macroscopic aspects of the system through statistical physics.



### [Parallel computing](https://en.wikipedia.org/wiki/Parallel_computing)

Since the calculation of forces is local, parallelization of molecular dynamics is relatively simple.

- Shared memory: Multiple processors (cores) share the same memory. Work is distributed across the cores using threads.

- Distributed memory: Each processor (node) has its own memory.

- However, program-multiple-data (SPMD)

- Data needs to be shared intermittently with other nodes.

- **Domain decomposition** (breaking the simulation box into physical domains) is feasible, especially for short-range interactions.



> **Domain decomposition**: Each processor owns a portion of the simulation domain and the atoms within it. The processor's domain is also extended to include required ghost atoms (**Ghost atoms**, which are copies of atoms located on other processors).



### Common molecular dynamics software

| Software                          | Copyright                |
|:---------------------------------:|:------------------------:|
| GROMACS                           | Open Source              |
| [LAMMPS](https://www.lammps.org/) | <mark>Open Source</mark> |
| AMBER                             | Commercial               |
| NAMD                              | Open Source              |
| DL_PLOY                           | Commercial               |
| Materials Studio                  | Commercial               |



## Basic steps of molecular dynamics

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/2d966fadc263491e9a0d950e22d1f34c/5a565f72-08a7-4fa4-9976-6edbd475de95.png)

### 1. Input information：

- Initial position and velocity

- Boundary conditions

- Interatomic potential



#### 1.1 Position and velocity

Initial position: randomly generated;

Initial velocity: typically obtained using a Boltzmann or Gaussian distribution. Before calculation, it's common to check that **the total momentum of the particles is zero**; otherwise, the system's inherent motion can lead to unstable total energy.

**Maxwell-Boltzmann rate distribution:**

$$
f(v)=\frac{4}{\sqrt{\pi}}\left(\frac{m}{2 k T}\right)^{3 / 2} v^2 e^{-\frac{m v^2}{2 k T}}
$$

Example: Generating random atomic coordinates using Python

```python
#!/usr/bin/env python
def genRandomAtoms(natoms, atom_types = 1, system_size = 10, outputFileName = 'randomAtoms.dat'):
    ......
    # Generate random atom positions
    dim = 3 # 3 dimensional
    minDist = 0.3 # minimum required distance between atoms
    positions = np.zeros((natoms,dim))
    positions[0] = np.random.rand(dim)
    for i in range(1,natoms):
        iter, maxIter = 0, 1e5
        while True and iter<maxIter:
            iter += 1
            newpos = np.random.rand(dim)*system_size
            dist = newpos-positions[0:i]
            if np.all( np.linalg.norm(dist,axis=1) > minDist ):
                break
        assert(iter<maxIter)
        positions[i]= newpos
```

#### 1.2 Boundary conditions

- Periodic boundary conditions: crystals, periodic flow systems

- Wrapping boundary conditions: isolated macromolecules, nanoparticles

- Fixed boundary conditions: thin film growth



#### 1.3 Interatomic potential

> **Interatomic potential** is a function or model that describes the interaction relationship between atoms. It is the most critical technology in atomic simulation and directly determines the reliability of atomic simulation results.

Reference：[Interatomic Potentials Repository (nist.gov)](https://www.ctcms.nist.gov/potentials/)



$$
V(\boldsymbol{r})=\sum_{\mathrm{ij}} V_{\mathrm{ij}}\left(\boldsymbol{r}_{\mathrm{i}}, \boldsymbol{r}_{\mathrm{j}}\right)+\sum_{\mathrm{ijk}} V_{\mathrm{ijk}}\left(\boldsymbol{r}_{\mathrm{i}}, \boldsymbol{r}_{\mathrm{j}}, \boldsymbol{r}_{\mathrm{k}}\right)+\sum_{\mathrm{ijkl}} V_{\mathrm{ijkl}}\left(\boldsymbol{r}_{\mathrm{i}}, \boldsymbol{r}_{\mathrm{j}}, \boldsymbol{r}_{\mathrm{k}}, \boldsymbol{r}_{\mathrm{l}}\right) \ldots
$$



Classification by **the number of multi-body interactions**:

- Two-body potentials: Lennard-Jones potential, Morse potential

- Three-body potential: Tersoff potential

- Multi-body potentials: Embedded atom potential, reaction force field, and **machine learning potential**

Machine learning potentials are driven by first-principles data. Potential functions other than these are also called **semi-empirical potentials**.

In practical applications, the appropriate interatomic potential should be selected based on the **bonding type** of the simulation object.



### 2. Time step evolution

#### 2.1 Force calculation

##### 2.1.1 Neighbor List

During molecular dynamics, a neighbor list is usually maintained to speed up the calculation of atomic forces.

```python
for I in particles:
    for J in nn_List[I]:
        force[I] += F(I, J)
        ......
```

##### 2.1.2 Interatomic interactions

$$
V(r)=4 \varepsilon\left[\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^6\right]
$$

The former is the Pauli exclusion principle, representing short-range repulsion; the latter is London dispersion, representing long-range attraction.

Furthermore, interatomic interactions include not only bonding but also non-bonding interactions.

#### 2.2 Time step integration

##### 2.2.1 Dynamic evolution

Molecular dynamics simulations involve two aspects:

- Configuration optimization

  In molecular dynamics, finding the lowest potential energy state of a system typically involves a large number of variables and a complex objective function, requiring specialized optimization algorithms such as steepest descent and conjugate gradient.

- Dynamic evolution

  For a molecular system composed of N atoms, the Newtonian equation for the motion of the $i$th atom is:
  
  $$
  f_i=m_i \frac{d^2 r_i}{d t^2}=m_i \bar{r}_i \cdots(\mathrm{i}=1,2,3, \ldots, \mathrm{N})
  $$
  
  Integrating its velocity gives the particle's **position trajectory**:
  
  $$
  r(t)=\int_0^t v(t) d t \longrightarrow\left\{\begin{array}{l}r(t+\Delta t)=r(t)+v(t) \Delta t+O\left(\Delta t^2\right) \\ v(t+\Delta t)=v(t)+\frac{F(t)}{m} \Delta t+O\left(\Delta t^2\right)\end{array}\right.
  $$
  
  Algorithms are used to determine the atomic velocities and positions at the next moment. These algorithms primarily include:

  - Euler algorithm

  - Velert algorithm

  - Leap-frog algorithm

  - Velocity-Velert algorithm

##### 2.2.2 Step size selection

Simple principle: The step size $\Delta t$ must be chosen to ensure energy conservation and time reversibility.

Complex requirement: When the system contains multiple time scales, such as mixed particles of varying masses, solvent aggregates, or coexisting flexible and rigid molecules, the step size must be chosen based on the component or model with the **fastest dynamics**.

| Motion                          |     Time Scale (sec)     |
|:------------------------------- |:------------------------:|
| Bond stretching                 | $10^{-14}$ to $10^{-13}$ |
| Elastic vibrations              | $10^{-12}$ to $10^{-11}$ |
| Rotations of surface sidechains | $10^{-11}$ to $10^{-10}$ |
| Hinge bending                   | $10^{-11}$ to $10^{-7}$  |
| Rotations of buried side chains |      $10^{-4}$ to 1      |
| Allosteric transistions         |      $10^{-5}$ to 1      |
| Local denaturations             |     $10^{-5}$ to 10      |

- Dynamic simulations are limited to the maximum frequency.

- The time step should be approximately 1/10 of the maximum frequency.

**Time step**: 1 fs (10-15 s).



#### 2.3 Temperature and pressure control

In actual molecular dynamics simulations, energy-conserving systems are very rare:

1. The energy-conserving NVE ensemble often does not accurately reflect the experimental conditions of the system;

2. Accumulated errors in molecular dynamics algorithms tend to cause the system to heat up.

Excess heat can be absorbed by connecting the system to a **heat bath**.



> An [**Ensemble**](https://en.wikipedia.org/wiki/Ensemble_(mathematical_physics) "In statistical physics, an ensemble represents a collection of a large number of possible states of a system under certain conditions. In other words, an ensemble is a probability distribution of the system's states.") is a collection of microscopic states that share the same macroscopic conditions (e.g., volume, energy, temperature, etc.). These microscopic states can be considered as a possible state of the system at a given moment. By studying the statistical behavior of these microscopic states, the average macroscopic properties of the system can be derived.



#### 2.4 Sampling and equilibrium

##### 2.4.1 Equilibrium process

> When a system is initialized or a simulation parameter changes, it is typically in a non-equilibrium state. During this period, the system's properties drift over time, tending toward a new equilibrium state. This process is called **equilibrium convergence**. After a period of time, when the system consistently maintains a certain energy value, we say it has reached equilibrium. The time required to reach equilibrium is called **relaxation time**.

##### 2.4.2 Sampling process

Molecular dynamics methods use **statistical averaging** to determine physical quantities in equilibrium systems. Data are collected during the **post-equilibrium simulation process** and then statistically analyzed.



### 3. Post-processing process

#### 3.1 Attribute calculation

MD is a **statistical mechanics** method: it generates a set of configurations that conform to a statistical distribution and calculates the ensemble average to obtain macroscopic measurements of physical quantities (long-term averages). We are generally not interested in the trajectories of individual atoms, but rather in thermodynamic quantities and macroscopic properties obtained by averaging over all atoms and time.


> [**Ergodic hypothesis**](https://en.wikipedia.org/wiki/Ergodic_hypothesis): For a system in equilibrium, the time average of a physical quantity is equal to the average of all systems in the corresponding ensemble.
> 
> - Time average:
> 
>$$
\overline{A}=\lim\limits_{\tau\to\infty}\frac{1}{\tau}{\int}_{\iota_0}^{\iota_0+\tau}A(\mathbf{q},\mathbf{p},t)\mathrm{d}t
$$
> 
> - Ensemble average:
> 
> $$
\langle A\rangle=\sum_\text{all states}{ A _ r p _ r}
$$
> 
> The ensemble average $<A>$ in statistical mechanics is the weighted average of the mechanical quantity $A$ over all realizable microstates.
> 
> When the thermodynamic system conforms to the **ergodic hypothesis** or the **quasi-ergodic hypothesis**, the average values ​​calculated by the above two methods are exactly the same.

Physical quantities can be broadly categorized as follows:

- Structural characterization

  Radial distribution function, dynamic structure factor, etc.

- Equation of state

  Free energy function, phase diagram, thermal expansion coefficient, etc.

- Transport properties

  Viscosity, thermal conductivity, correlation function, diffusion coefficient, etc.



| Property    | Definition                                                                                                                                                                                                                                  | Application                                    |
|:-----------:|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|:----------------------------------------------:|
| Temperature | $T=\frac{1}{3} \frac{1}{N k_B}<\sum_{i=1}^N m_i \vec{v}_i^2>\quad \vec{v}_i^2=\vec{v}_i \cdot \vec{v}_i$                                                                                                                                      | Direct                                         |
| Pressure    | $P=\frac{1}{3 V}<\sum_{i=1}^N\left(m_i \vec{v}_i^2+\vec{r}_i \cdot \vec{f}_i\right)>$                                                                                                                                                         | Direct                                         |
| Stress      | $\sigma_{i j}=\frac{1}{\Omega}<\left(-\sum_\alpha m_\alpha u_{\alpha, i} u_{\alpha, j}+\left.\frac{1}{2} \sum_{\alpha, \beta, a \neq \beta} \frac{\partial \phi(r)}{\partial r} \frac{r_i}{r} \cdot r_j\right\|_{r=r_{\alpha \beta}}\right)>$ | Direct                                         |
| MSD         | $<\Delta r^{2}(t)>=\frac{1}{N} \sum_{i}\left(r_{i}(t)-r_{i}(t=0)\right)^{2}$                                                                                                                                                                  | Diffusivity                                    |
| RDF         | $g(r)=<\frac{N\left(r \pm \frac{\Delta r}{2}\right)}{\Omega\left(r \pm \frac{\Delta r}{2}\right) \rho}>$                                                                                                                                      | Atomic structure (signature)                   |
| VAF         | $<v(0)v(t)>=\frac{1}{N}\sum_{i=1}^{N}\frac{1}{N_{i}}\sum_{k=1}^{N_{i}}v_{i}(t_{k} )v_{i}(t_{k}+t)$                                                                                                                                            | Diffusivity, phase state, transport properties |



#### 3.2 Analysis and visualization



## Advantages and Limitations of Molecular Dynamics

- Advantages:

  - It interfaces well with first-principles calculations (quantum mechanics) and finite element methods (continuum mechanics)

  - It provides detailed information at the atomic and molecular levels

  - It is applicable to a wide range of fields and can simulate a variety of systems and conditions

**MD is a deterministic process**: given an initial position and velocity, the time evolution of a system is, in principle, completely deterministic. A major advantage of MD is its atomic-level resolution for studying **rapid nonequilibrium processes** (such as damage/plastic deformation due to shock waves, dynamic fracture and crack growth, ion bombardment, cluster impacts, and so on). For many of these problems, there is no alternative but MD.



- Molecular Dynamics Applicability:

In molecular dynamics simulations, the Schrödinger equation for the nucleus is replaced by Newton's equations.

A key criterion for the validity of this approximation is the **de Broglie wavelength**: when the de Broglie wavelength is much larger than the interparticle spacing, quantum effects become very significant. Therefore, under most sufficiently high temperature conditions, except for light atoms such as $H$, $He$, and $Ne$, all other atoms can be approximated as point masses ($d>>\Lambda$), and their motion can be described using Newton's equations.

- Molecular Dynamics Simulation Time and Size

  - Time: ~nanoseconds (1-100 ns)

  - Size: ~$10^8$ atoms

# Key factors in molecular dynamics

---


## 1 Periodic Boundary Condition (PBC）

- The system repeats itself periodically at its boundaries.

- Atoms escaping from a boundary reappear at the corresponding position on the adjacent boundary.

Tiling a small unit cell into an infinitely large bulk material.

$$
A(x) = A(x + nL)
$$

**Eliminate surface effects** and construct a quasi-infinite volume to more accurately represent the macroscopic system

### 1.1 Instances

Bravais lattice：

$$
aP、mP、mC、oP、oC、oI、oF、rP=hR、tP、tI、hP、cP、cI、cF
$$

### 1.2 Basic Features

- Translational symmetry

- The boundary is fixed, allowing atoms to interact across it.

- The number of atoms in the system remains constant.
  

### 1.3 Code Implementation Outline

#### 1.3.1 Crossing the boundary

```python
position = (position - box.lower) % L + box.lower
```

#### 1.3.2 Interatomic distance

Periodic boundary conditions result in an infinite number of mirror images of the simulated system. The force on a particle is theoretically the force on all particles in all mirror images!

**Minimum Image Convention**: When combining **potential field truncation** with **periodic boundary conditions**, it is usually required that <mark>the net interaction experienced by each atom only includes that from one (atom/mirror image) in the system</mark>. In molecular dynamics, it is usually required that the unit cell length $L>2r_{cut}$

#### 1.3.3 Important objects

- `class Particle` ：particles
  
- `class Box` ：box
  
- `class BoundaryCondition` ：boundary conditions
  
  - `class PeriodicBoundaryCondition` ：periodic boundary conditions

`class Particle` encapsulates the particle position information and provides an interface for getting and setting the position information.

```python
class Particle:
    def __init__(self, position):
        self._position = np.array(position,dtype=float)

    @property
    def position(self):
        return self._position

    @position.setter
    def position(self, value):
        self._position = np.array(value)
```

`class Box` encapsulates the upper bound, lower bound, and length information of the box, and can obtain the box length through string methods.

```python
class Box:
    def __init__(self, lower, upper):
        self.lower = np.array(lower, dtype=float) 
        self.upper = np.array(upper, dtype=float) 

    @property
    def length(self):
        return self._upper - self._lower
```

`class BoundaryCondition` is a base class used to implement different boundary conditions. Subclasses that inherit this class must implement the `apply` and `displacement` functions.

```python
class BoundaryCondition:
    """Abstract base class for boundary conditions."""

    def apply(self, particles, box):
        raise NotImplementedError

    def displacement(self, position1, position2, box):
        raise NotImplementedError
```

`class PeriodicBoundaryCondition` is a subclass of `class BoundaryCondition`.

```python
class PeriodicBoundaryCondition(BoundaryCondition):
    ......
    def apply(self, particles, box):
        for p in particles:
            p.position = (p.position - box.lower) % box.length + box.lower


    def displacement(self, position1, position2, box):
        dr = position1 - position2 
        dr = dr - np.rint(dr / box.length) * box.length
        return dr
```

## 2 Finite distance cutoff of potential field

### 2.1 Interatomic forces

$$
V(R)=\sum_{i<j}\nu(\mid r_i-r_j\mid)
$$

Assumptions:

- The molecule is rigid and in a **nondegenerate ground state**;

- The interactions between atoms are weak, so the internal structure of the molecule is less affected by the environment;

- There is a short-range repulsion between atomic nuclei.

Main characteristics of the two-body potential:

$$
V(r)=4 \varepsilon\left[\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^6\right]
$$

- Repulsion at close distances: Hugonoit equation

- Attraction near the bond length and beyond: Elasticity, dielectric experiments, and thermal expansion experiments

- In practical applications, a cutoff radius is often chosen

  - Computational complexity before truncation: $O(N^2)$

  - Computational complexity after truncation: $O(mN)$
    

### 2.2 Theoretical Basis of Potential Field Truncation

- The Lennard-Jones potential (and similar two-body potentials) **decays rapidly with distance**.
  
  Before truncation:
  
  $$
  V(r)=4 \varepsilon\left[\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^6\right]
  $$
  
  After truncation:
  
  $$
  V(r)=\begin{cases}4\varepsilon\left[(\frac{\sigma}{r})^{12}-(\frac{\sigma}{r})^6\right]&\quad r\leq r_c\\0&\quad r>r_c\end{cases}
  $$

- Ignoring interactions outside the cutoff radius can **save a lot of computation time**
  

Error term caused by potential field truncation:

$$
U_{tail}=\frac{N\rho}2\int_{r_c}^\infty drV(r)4\pi r^2
$$

When the potential field $V(r)$ decays slower than $r^{-3}$ (such as the Coulomb potential $V(r)\sim\frac{1}{r}$), the error term caused by truncation will diverge!

$$
\begin{aligned}&V(r){\sim}\frac1{r^2}\quad\longrightarrow\quad U_{tail}\sim r\\\\&V(r){\sim}\frac1{r^4}\quad\longrightarrow\quad U_{tail}\sim\frac1r\end{aligned}
$$

In most cases, we can use the **shielded Coulomb potential**:

$$
V(r){\sim}\frac1r\exp(-\kappa r)
$$

The potential field after truncation

$$
V(r)=\begin{cases}4\varepsilon\left[(\frac{\sigma}{r})^{12}-(\frac{\sigma}{r})^6\right]&\quad r\leq r_c\\0&\quad r>r_c\end{cases}
$$

At $r=r_c$, $V(r)$ is discontinuous, the force $F=-\frac{dV(r)}{dr}$ must diverge, and the molecule is dynamically unstable!

### 2.3 Outline of potential field truncation method

#### 2.3.1 Truncated and Shifted Potential

$$
V_0(r)=\begin{cases}V(r)-V(r_c),\quad&r<r_c\\0,\quad&r\geq r_c&\end{cases}
$$

When $r=rc$, **the first-order derivative of the energy becomes discontinuous!**

The shifted potential **well depth** also deviates from the original potential function, causing errors in the calculation of thermodynamic properties.

#### 2.3.2 Switch Function

Introduce the **switching function** $S(r)$:

$$
\tilde{V}(r)=V(r)\color{red}{S(r)}
$$

Require:

$$
\begin{aligned}S(r_s)&=1&&S'(r_s)&=0\\\\S(r_c)&=0&&S'(r_c)&=0\end{aligned}
$$

- $\cos$ switching function:
  
  $$
  \begin{gathered}
  t=\frac{r_c-r}{r_c-r_s}=\begin{cases}1&r=r_s\\0&r=r_c&\end{cases} \\
  \begin{aligned}S(t)=\color{red}{\frac{1}{2}}(1-cos\pi t)\end{aligned} 
  \end{gathered}
  $$
  
- Polynomial switching function:
  
  $$
  \begin{gathered}
  t=\frac{r_c-r}{r_c-r_s}=\begin{cases}1&r=r_s\\0&r=r_c&\end{cases} \\
  S(t)=\color{red}{t^3(10-15t+6t^2)} 
  \end{gathered}
  $$
  

## 3 Neighbor List

In principle, the interaction between pairs of atoms has a non-zero value regardless of the distance between them. However, in practice, for potentials like the Lennard-Jones, the interaction between atoms is approximately zero once the distance between them exceeds a few times $\sigma$. Therefore, **the number of neighboring atoms that interact with a given atom is usually limited, and this can be stored using a neighbor list.**

### 3.1 Construction and form of neighbor list

Neighbor List:

- Checks if particles fall within each other's radius

- Only needs to **occasionally rebuild the neighbor list**
  

#### 3.1.1 Form of neighbor list

| 1   | 2   | 3   | 4   | 5   | 6   | ... |
| --- | --- | --- | --- | --- | --- | --- |
| 6   | 45  | 9   | 15  | 19  | 1   |     |
| 34  | 57  | 16  | 43  | 31  | 3   |     |
| 17  | 16  | 25  | 36  | 14  | 19  |     |
| 29  | 10  | 61  | 13  | 59  | 40  |     |
| 12  | 8   | 58  | 7   | 63  | 21  |     |
| 18  |     | 12  | 27  | 26  | 64  |     |
| 47  |     | 30  | 38  |     | 39  |     |
| 19  |     | 21  |     |     | 23  |     |
|     |     | 51  |     |     | 14  |     |

#### 3.1.2 Role of the neighbor list

- Without neighbor list:
  

```python
for I, J in particles:
    if r(I, J) < Rcut:
        potential[I] += pot(I, J)
        force[I] += F(I, J)
        ......
```

- With neighbor list:
  

```python
for I in particles:
    for J in nn_List[I]:
        potential[I] += pot(I, J)
        force[I] += F(I, J)
        ......
```

#### 3.1.3 Nearest Neighbor Algorithm

- Brute force enumeration
  
  ```python
  for I, J in particles:
      if r(I, J) < Rcut:
          nn_List[I].append(J)
          ......
  ```
  
- VerletList method（Reference: [*Phys. Rev.* **159**, 98 (1967)](https://journals.aps.org/pr/abstract/10.1103/PhysRev.159.98)）
  
  ![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/1132613cd2ff41549786fed20fa46d0a/d69a3743-2cce-4aa3-bc2b-935e6c20fdcb.png)
  
  - For each atom, maintain a list of neighboring atoms within the range $r_s=r_c+\delta$

  - If $r_{ij}<r_c$, calculate the force

  - Update the neighbor list every several time steps (typically about 10-20 time steps)
    

  The appropriate $\delta$ depends on the system's temperature, diffusion rate, density, and other factors.

  The criterion for updating VerletList is: the maximum displacement of atoms in the system is greater than half the shell thickness $\color{red}\delta$.

  Code implementation:
  
  ```python
  class VerletList(NeighborList):
      ......
      def build(self):
          self.neighbor_list = {}
          self.previous_positions = {p: np.copy(p.position) for p in self.particles}
  
          for p1 in self.particles:
              self.neighbor_list[p1] = []
              for p2 in self.particles:
                  displacement = self.boundary_condition.displacement(p1.position, p2.position, self.box)
                  if p1 != p2 and np.linalg.norm(displacement) < (self.cutoff + self.skin_depth):
                      self.neighbor_list[p1].append(p2)
  
      ......
      def update(self):
          max_displacement = max(np.linalg.norm(self.boundary_condition.displacement(p.position, self.previous_positions[p], self.box)) for p in self.particles)
          if max_displacement > self.skin_depth / 2:
              self.build()
  ```
  
  `class VerletList` provides an interface for getting and setting the neighbor list. The `build` function is used to build the neighbor list, and the `update` function is used to determine whether the neighbor list needs to be updated.
  
- Interval sorting (CellList)

  - When calculating the neighbor list, divide the box into cells of size $L=R_c$

  - When constructing the neighbor list, only the 26 neighboring cells of the particle are considered.

  - The complexity of constructing the neighbor list is $O(N)$
    

## 4 Temperature and pressure control

### 4.1 Temperature

- **Energy** in molecular dynamics is typically normalized by the number of particles in the system, and the total energy of the system is linearly related to the number of particles.

- The temperature of a system can be measured using **the average kinetic energy of the particles**.
  

$$
T(t)=\frac1N\sum_{i=1}^N\frac12mv_i(t)\cdot v_i(t)
$$

Typically, during a molecular dynamics simulation, the temperature of the system fluctuates. To achieve temperature control, we need to introduce an additional motion control, namely a **heat bath**.

**Temperature Control**:

A temperature control mechanism can maintain a system's temperature at a given value, or it can cause the system's temperature to fluctuate depending on the ambient temperature. A reasonable temperature control mechanism can produce a correct statistical ensemble, meaning that the probability of each particle configuration occurring after temperature control satisfies the laws of statistical mechanics.

- Stochastic Methods

  - Anderson Thermal Bath

  - Langevin Thermal Bath

  - Dissipative Particle Dynamics

- Deterministic Methods

  - Direct Velocity Calibration

  - Berendsen Temperature Control

  - Nosé-Hoover Temperature Control
    

#### 4.1.1 Direct velocity calibration

$$
T(t)=\sum_{i=1}^N\frac{m_iv_i^2(t)}{k_BN_f}
$$

Introduce speed calibration factor:

$$
\lambda=\sqrt{\frac{T_\mathrm{c}}{T(t)}}
$$

At regular integration steps, the velocity is calibrated by replacing $v(t)$ with $\lambda v(t)$, causing the system temperature to fluctuate slightly around the target value.

- Advantages: Simple principle, easy to program

- Disadvantages: The simulated system cannot correspond to the ensemble of statistical mechanics; abrupt velocity calibration causes abrupt changes in the system's energy, which deviates significantly from the equilibrium state of the actual structure.
  

#### 4.1.2 Nosé-Hoover Temperature Control

By introducing an additional "virtual" degree of freedom (a heat bath) that interacts with the system's particles to absorb or release energy, the system's temperature is brought closer to the target value.

Basic idea: **Adjusting the particle velocity and average kinetic energy in the system by varying the time step**. Therefore, in the Nosé-Hoover method, a new variable $s$ is introduced to readjust the time unit.

- Mapping a virtual system evolving according to microscopic canonical law to a real physical system evolving according to a canonical ensemble

- Proving that the microscopic canonical distribution in the virtual system is equivalent to the canonical distribution of the $(p', r')$ variable in the real system.
  

### 4.2 pressure

In molecular dynamics, pressure is defined as the average force per unit area on the inner wall of a container. **The microscopic source of pressure is the transfer of momentum per unit area.**

$$
P=\frac1{3V}[\sum_im_iv_i^2+\sum_i\sum_jr_{ij}\cdot f_{ij}]
$$

1. Atoms carry momentum across the unit plane.

2. Atoms on both sides of the unit plane interact with each other.
  

In MD, pressure control is typically achieved by **changing the size or shape of the simulation box**:

- Commonly used pressure control algorithms include Berendsen pressure coupling, the Parrinello-Rahman method, and the Nosé-Hoover pressure coupling.

- Periodic boundary conditions are required on surfaces in the direction of pressure control to minimize surface effects and ensure simulation accuracy.

- Atoms or molecules in a system often rearrange to adapt to varying pressure conditions, and this rearrangement can sometimes lead to **the formation of new stable or metastable crystal structures**.

- Pressure control does not consider the local nature of interactions and exhibits certain unphysical dynamic characteristics.
  

**References：**

1. [*J. Chem. Phys.* **72**, 2384-2393 (1980)](https://pubs.aip.org/aip/jcp/article/72/4/2384/218722/Molecular-dynamics-simulations-at-constant)
  
2. [*J. Appl. Phys.* **52**, 7182-7190 (1981)](https://pubs.aip.org/aip/jap/article/52/12/7182/9558/Polymorphic-transitions-in-single-crystals-A-new)
  
3. [*Phys. J. B* **79**, 241-252 (2011)](https://link.springer.com/article/10.1140/epjb/e2010-10763-x)

# Interatomic Potentials

---


> **Interatomic potential** is a function or model that describes the interaction relationship between atoms. It is the most critical technology in atomic simulation and directly determines the reliability of atomic simulation results.

Ref：[Interatomic Potentials Repository (nist.gov)](https://www.ctcms.nist.gov/potentials/)

## 1 Lennard-Jones Potential

$$
V(r)=4 \varepsilon\left[\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^6\right]
$$

The former is the Pauli exclusion principle of short-range repulsion; the latter is the London dispersion of long-range attraction.

### 1.1 Physical explanation

#### 1.1.1 Attractive term

$$
U_{12}=-\vec{p}_2\cdot\vec{E}_{12}\propto-E_{12}^2\propto-\frac1{r_{12}^6}
$$

The Hamiltonian of a system consisting of two interacting oscillators is

$$
\mathcal{H}= \mathcal{H}_1+\mathcal{H}_2+\Delta\mathcal{H}_{12}
$$

The **perturbation term** is the interaction between the two dipoles

$$
\Delta\mathcal{H}_{12}=\frac{\vec{p}_1\cdot\vec{p}_2-3(\vec{n}\cdot\vec{p}_1)(\vec{n}\cdot\vec{p}_2)}{r_{12}^3}
$$

From **first-order perturbation theory** we can calculate the energy change

$$
\Delta E\simeq\sum_n\frac{|\langle\psi_n|\Delta\mathcal{H}_{12}|\psi_0\rangle|^2}{E_0-E_n}\propto-\frac1{r_{12}^6}
$$

#### 1.1.2 Repulsive term

**Interatomic interactions**: Intramolecular chemical bonds (bond length, bond angle, torsion, etc.), intermolecular van der Waals forces, hydrogen bonds, polarization, etc.

**Main sources of interatomic repulsion**:

- **Van der Waals repulsion**: When two atoms are too close, the repulsive effect between their electron clouds becomes significant;

- **Coulomb force**: The repulsive force between electrons and between protons

Among them, $\frac1{r^{12}}$ has no scientific basis.

### 1.2 Force expression

$$
\begin{gathered}
F_{1,\alpha}=-\frac{\partial V}{\partial r}\frac{\partial r}{\partial r_{1,\alpha}} \\
 \\
\begin{aligned}\frac{\partial V}{\partial r}=4\varepsilon\left[\frac{-12}r(\frac\sigma r)^{12}+\frac6r(\frac\sigma r)^6\right]\quad\quad\frac{\partial r}{\partial r_{1,\alpha}}=\frac{\vec{r_\alpha}}r\end{aligned} \\
F_{1,\alpha}=48\frac{\varepsilon}{r}\bigg[(\frac{\sigma}{r})^{12}\!\!-0.5(\frac{\sigma}{r})^{6}\bigg]\frac{\bar{r_{\alpha}}}{r} 
\end{gathered}
$$

where $\vec{r}=\vec{r}_1-\vec{r}_2$，$r=|\vec{r}_1-\vec{r}_2|$.

### 1.3 Equilibrium distance between atoms

When $F_{1,\alpha}=0$, it is equivalent to $\frac r\sigma=2^{1/6}\approx1.122$,

$$
V_{min}(r)=4\varepsilon\left[(\frac1{\sqrt[6]{2}})^{12}-(\frac1{\sqrt[6]{2}})^6\right]=-\varepsilon 
$$

### 1.4 Reduced units

For the Lennard-Jones potential, it is common to use the **model's intrinsic length and energy units** to characterize the relevant quantities, i.e., using $\sigma$ to measure length and $\epsilon$ to measure energy. We call this unit system the **L-J unit**.

$$
\begin{aligned}
&r_i^{\prime}=r_i/\sigma &&&&& V^{\prime}(r)=V(r)/\varepsilon   \\
&\text{原始公式：}&&&&& V(r)=4\varepsilon\left[(\frac{\sigma}{r})^{12}-(\frac{\sigma}{r})^6\right]  \\
&\text{约化公式：}&&&&& V^{\prime}(r)=4[\frac1{r^{\prime\left.12\right.}}-\frac1{r^{\prime6}}] 
\end{aligned}
$$

**Motion equation in SI**：

$$
m\frac{d^2}{dt^2}\boldsymbol{r}_i=\sum_j48\varepsilon_0[0.5\left(\frac\sigma{r_{ij}}\right)^6-\left(\frac\sigma{r_{ij}}\right)^{12}](\boldsymbol{r}_{ij}/{r_{ij}}^2)
$$

Substituting $\mathbf{r}_i^{\prime}=\mathbf{r}_i/\sigma$, $t^{\prime}=t/\tau$, $\tau=\sigma\sqrt{m/\varepsilon}$, we get

**Motion equation in L-J unit**：

$$
\frac{d^2}{d(t^{\prime})^2}{\boldsymbol{r}_i}^{\prime}=\sum_j24(r_{ij}^{-6}-2r_{ij}^{-12})({\boldsymbol{r}_{ij}}^{\prime}/{r_{ij}}^{\prime2})
$$

*This differential equation has nothing to do with the specific properties of the noble gas molecules*.

## 2 Other types of two-body potential

### 2.1 Simple harmonic potential

$$
\begin{aligned}V(r)&=\frac12k(r-r_0)^2\\\vec{\boldsymbol{F}}(r)&=(\mathrm{r}-\mathrm{r}_0)\frac{\vec{\boldsymbol{r}}}r\end{aligned}
$$

- Simple harmonic potentials are typically suitable for describing **small-amplitude molecular vibrations**, i.e., small-amplitude vibrations of atoms around their equilibrium positions.

- Simple harmonic potentials can describe the **bond stretching** of two covalently bonded atoms.

### 2.2 Hard sphere model

$$
\mathrm{V}(r)=\begin{cases}\quad0&\quad r\geq2R\\+\infty&\quad r<2R&\end{cases}
$$

- The hard-sphere model treats molecules as infinitely small hard spheres with a hard core radius.

- **Interactions between molecules are described solely by repulsive forces**.

  - The potential energy function in the hard-sphere model is typically zero.

  - When two spheres are in contact, the potential energy is infinite.

### 2.3 Soft sphere model

$$
\mathrm{V}(r)=\varepsilon(\frac\sigma r)^n
$$

In fact, when molecules are close to each other, **the electron clouds can overlap to a certain extent**, so there can be a certain degree of penetration between molecules. This is the soft ball model.

> The **hard sphere model** and **soft sphere model** lack consideration of the **attractive effects** between molecules and therefore cannot be used to simulate real fluids, but they can be used to examine the effects of repulsive effects on fluid structure and liquid-solid equilibrium.

### 2.4 Buckingham potential

$$
\mathrm{U}(r)=Ae^{-Br}-\frac C{r^6}
$$

The first term is the short-range repulsive force, and the second term is the long-range attractive force.

- Compared with the Lennard-Jones potential, **the repulsive term in the Buckingham potential is more flexible and practical**, but also significantly more computationally intensive.

- When $\mathrm{r\to0}$, because the exponential term converges to a constant, **the Buckingham potential becomes unstable as the potential function diverges with decreasing interatomic distance**.

### 2.5 Born-Mayer potential

$$
\mathcal{U}(r)=Ae^{-\beta r}-\frac{kq_iq_j}r
$$

The Born-Mayer potential is used to describe the interactions between **ions in a crystal lattice**. Compared to the simple Coulomb model, the Born-Mayer model takes into account **the short-range repulsions** arising from the repulsive Pauli exclusion principle, making it a more modern and accurate representation of ionic interactions.

### 2.6 Morse potential

$$
\mathcal{U}(r)=-D_e+D_e[1-e^{-\alpha(r-r_e)}]^2
$$

The Morse potential is a simple analytical model for the potential energy between **diatomic molecules**.

Taylor expansion of the Morse potential around $r_e$ yields

$$
\mathrm{V}(r)\approx\frac12\mathrm{~}k(r-r_0)^2
$$

- $D_e$ is the potential energy depth, representing the value at the lowest point of potential energy.

- $\alpha$ is a parameter that adjusts the steepness of the potential energy curve and can be used to fit other material parameters such as cohesive energy and lattice constant.

- $r_e$ is the equilibrium distance of the potential energy curve.

### 2.7 Coulomb potential

$$
V_{elec}=C_{unit}\sum_{i<j}\frac{q_iq_j}{R_{ij}}
$$

Coulomb interactions **decay very slowly**.

Methods for obtaining **atomic charges**:

- Formal charge

- Dipole moment or polarization

- Electronic structure calculation

- Self-consistent charge balance

Under periodic boundary conditions (infinite systems), the Coulomb potential is **conditionally convergent**. The result depends on **the order of addition** and **the surface charge arrangement**, requiring specialized summation techniques and boundary conditions, such as the Ewald method and the particle-mesh Ewald technique.

### 2.8 Yukawa potential

$$
\mathrm{V}(r)=-g^2\frac{e^{-\kappa mr}}r
$$

The Yukawa potential describes **the short-range interaction between nucleons**, where $g$ is the [coupling constant](https://en.wikipedia.org/wiki/Coupling_constant) for the particle interaction; $m$ is the mass of the interacting particle; $r$ is the radial distance from a point around the particle to the particle; and $\frac1{\kappa m}$ represents the range of the nuclear force.

When $e^{-\kappa mr}==1$, that is, $V={\frac{q}{4\pi\varepsilon_{0}}}{\frac{1}{r}}$, the Yukawa potential degenerates into a Coulomb potential, at which point the rest mass of the photon is zero.

### 2.9 Aziz potential

$$
\begin{aligned}&\mathrm{V}(r)=\varepsilon\left[Ae^{-\alpha x}-F(x)\sum_{j=0}^2\mathrm{C}_{2j+6}/x^{2j+6}\right]\\&F(x)=\begin{cases}e^{-(\frac Dx-1)^2}&x<D\\1&x\geq D&\end{cases}\quad x=\frac r{r_m}\end{aligned}
$$

The Aziz potential describes the **interactions of noble gases, such as helium and neon, at low and moderate temperatures**.

The difference between the Buckingham potential and the Aziz potential lies primarily in the smaller interatomic distances, with the Aziz potential exhibiting **a higher repulsive force** than the Buckingham potential.

### 2.10 Tang-Toennies potential

$$
V(r)=Ae^{-br}-\sum_{n=3}^Nf_{2N}(bR)\frac{C_{2N}}{R^{2N}}
$$

The Tang-Toennies potential is used to describe **the van der Waals interaction between noble gases (He and Rn)**. The first term is the intermolecular Coulomb repulsion potential, and the second term is the dispersion energy of the intermolecular interaction and the electron cloud overlap effect. $f_{2N}(x)$ is the decay function:

$$
f_{2N}(x)=1-e^{-x}\sum_{k=0}^{2n}\frac{x^k}{k!}
$$

## 3 Limitations of the two-body potential

Usually the total energy of a system can be divided into different contribution terms according to the coordinates of individual potential, pair potential, three-body potential, etc.:

$$
\mathcal{V}=\sum_iv_1(\vec{r}_i)+\sum_i\sum_{j>i}v_2(\vec{r}_i,\vec{r}_j)+\sum_i\sum_{j>i}\sum_{k>j>i}v_3(\vec{r}_i,\vec{r}_j,\vec{r}_k)+\ldots 
$$

The **pair potential approximation** provides a very good description of the total energy of a system. By defining the **effective pair potential**, it can partially include the average three-body effect. This effective pair potential is often used in computer simulations, representing some of the many-body effects.

$$
\mathcal{V}\approx\sum_iv_1(\vec{r}_i)+\sum_i\sum_{j>i}v_2^{\mathrm{eff}}(r_{ij})
$$

**The effective pair potential required to reproduce experimental data may depend on material density, temperature, and other factors**, whereas the true interatomic pair potential does not.

- Difficulty in describing many-body effects

  **In real material systems, interatomic interactions depend not only on bond lengths and bond energies but also on the chemical environment.**

- Difficulty in describing chemical environment dependence
  
  $$
  \begin{aligned}\phi_\mathrm{stretch}&=\frac{1}{2}k_\mathrm{stretch}(r-r_0)^2\\k_{\mathrm{stretch},sp^2}&\neq k_{\mathrm{stretch},sp^3}\end{aligned}
  $$
  
  Two-body potential parameter sets are only valid for certain molecule/bond types.

- Difficult to accurately predict stable configurations
  
  The problem with the two-body potential lies in its **form** itself, not in its **parameters**.

### 3.1 Pair potential calculation of vacancy formation energy

Assume

- Atomic coordination number is $Z$

- Limited to nearest-neighbor interactions

$$
\begin{aligned}E_{total}(N)&=\frac{1}{2}NZE_{bond}\\\\E_{cohesive}(N)&=\frac{E_{total}}{N}=\frac{1}{2}ZE_{bond}\end{aligned}
$$

Vacancy formation energy is defined as:

$$
\varepsilon_{vac}=E_{vac}(N-1)-\frac{N-1}NE_{total}(N)
$$

Substituting into the above formula, we get

$$
\varepsilon_{{vac}}=-\frac12ZE_{bond}
$$

**Under the pair potential approximation, the vacancy formation energy is numerically equal to the binding energy (with opposite signs)**, regardless of the specific form of the pair potential. However, in actual material systems, the ratio of the vacancy formation energy to the binding energy is $\sim\frac13$.

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/3bd95ca3deef421890ab510b1da57587/009244d2-48a9-47cc-8d7d-96d998024f59.png)

### 3.2 Atomic surface relaxation direction

Consider a one-dimensional infinite chain

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/3bd95ca3deef421890ab510b1da57587/24b8d143-9a9a-4345-833b-65e4d2ba2bc8.png)

$$
\begin{aligned}
&E=\frac{1}{2}\times2\times4\varepsilon\sum_{n=1}\left[\left(\frac{\sigma}{an}\right)^{12}-\left(\frac{\sigma}{an}\right)^6\right] \\
&=4\varepsilon\left[1.00\left(\frac\sigma a\right)^{12}-1.02\left(\frac\sigma a\right)^6\right]
\end{aligned}
$$

Solving for energy extremes

$$
\begin{aligned}&{a=(\frac2{1.02})^{1/6}\sigma\approx1.118\sigma}\\&{\varepsilon_\mathrm{c}=1.04\varepsilon}\end{aligned}
$$

It is in a **compressed** state compared to its equilibrium position.

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/3bd95ca3deef421890ab510b1da57587/7a90da7a-a118-4f53-bdb1-f1f69c0651eb.png)

$$
\begin{aligned}&r_n=na=n(\frac2{1.02})^{1/6}\sigma\\\\&(\frac\sigma{r_n})^6=(\frac{1.02}2)\frac1{n^6}\end{aligned}
$$

Calculate atomic surface forces:

$$
\begin{aligned}
\text{F}& =4\varepsilon\sum_{n=1}^\infty\left[\frac{12}{r_n}(\frac\sigma{r_n})^{12}-\frac6{r_n}(\frac\sigma{r_n})^6\right]\widehat{x}  \\
&=\frac{4\varepsilon}a\sum_{n=1}^\infty\left[\frac{12}{n^{13}}(\frac{1.02}2)^2-\frac6{n^7}(\frac{1.02}2)\right]\widehat{x} \\
&=0.1456\frac\varepsilon a\widehat{x}
\end{aligned}
$$

> - [*Riemann $\zeta$ function*](https://en.wikipedia.org/wiki/Riemann_zeta_function)
> 
> $$
\begin{aligned}\sum_{n=1}^\infty\frac1{n^{13}}&=1.00024\\\sum_{n=1}^\infty\frac1{n^7}&=1.00835\\\end{aligned}
$$

It can be seen that the surface atoms are subjected to a net outward force, and **will relax toward the outside of the surface under the action of the pair potential**, while in reality, the surface atoms usually relax toward the inside.

### 3.3 The root cause of the pair potential problem

- In the **pair potential model**, the bond energy between atoms is unaffected by the bonding of surrounding atoms.

- In practice, when the central atom bonds with more atoms, the average bond energy decreases.

  Pair potential prediction: $E\propto Z$

  Actual system: $E\propto \sqrt{Z}$

## 4 Many-body potential and machine learning potential

### 4.1 EAM potential

$$
\begin{aligned}E_\mathrm{coh}&=\underbrace{\sum_iF_i(\rho_i)}_{\text{Embedding energy}}+\underbrace{\frac{1}{2}\sum_i\sum_{j\neq i}V(R_{ij})}_{\text{Pair potential}}\\ \rho_i&=\sum_{i\neq j}f(R_{ij})\end{aligned}
$$

The [Embedded atom model (EAM）](https://en.wikipedia.org/wiki/Embedded_atom_model) is suitable for **interactions between atoms in metals and alloys**, such as Fe, Cu, Ni, Pt, and Au.

Core Ideas:

- The energy of an atom depends **nonlinearly** on its neighboring atoms (number and distance).

- **Electron density** is used as a measure of the local chemical environment.

Where the first term is **the embedding energy of the nucleus in the electron gas**, the second term is **the mutual repulsion between nuclei**.

### 4.2 MEAM potential

Since the electron density in the EAM potential is not directional, it was later extended to the Modified Embedded Atom Method (MEAM) potential.

The MEAM potential is similar in concept to the EAM potential, but **the electron density term retains different $s$, $p$, and $d$ components**:

$$
\begin{gathered}
E_{\mathrm{coh}} =\underbrace{\sum_iF_i(\rho_i)}_{\text{Embedding energy}}+\underbrace{\frac12\sum_i\sum_{j\neq i}V(R_{ij})}_{\text{Pair potential}} \\
\boldsymbol{\rho}_{i} =\sum_{j,k}f(R_{ij})\cdot f(R_{ik})\cdot g(\cos\theta_{ijk}) 
\end{gathered}
$$

The contribution of the **angular term** is particularly important for **transition metal elements** and **covalent systems**.

### 4.3 Molecular Reaction Force Field

- Bonding Interactions

  - Bonding interactions generally represent the properties of covalent bonds;

  - Bond stretching, bending, and torsion can be expressed as Hooke's law.

- Non-bonding Interactions

  Non-bonding interactions generally arise from electrostatic interactions, van der Waals interactions, and other factors.

Chemical bonding interactions include di-, tri-, and tetra-body interactions.

$$
\begin{aligned}U&=\frac12\sum_{i,j}V(R_i,R_j)+\frac16\sum_{i,j,k}V(R_i,R_j,R_k)+\ldots\end{aligned}
$$

- Two-body interactions:
  
  - Simple harmonic potential
    
    $$
    \phi_{bond}(R)=\frac12k_{\alpha\beta}\Big(R-R_{\alpha\beta}^0\Big)^2
    $$
    
    where $k_{\alpha\beta}\sim700kcal/mol/\AA$.
  
  - Lennard-Jones potential
    
    $$
    V(r)=4\varepsilon\Bigg[\Bigg(\frac{\sigma}{r}\Bigg)^{12}-\Bigg(\frac{\sigma}{r}\Bigg)^6\Bigg]
    $$
  
  - Morse potential
    
    $$
    \nu(r)=\varepsilon[e^{-2a(r-r_0)}-2e^{-a(r-r_0)}]
    $$
  
  - Buckingham potential

    $$
    \mathrm{U}(r)=Ae^{-Br}-\frac C{r^6}
    $$

- Three-body interaction
  
  - [ ] $E_\theta=\sum K_\theta(\theta-\theta_0)^2$
  
  - [x] $E_\theta=\frac12\frac{k_{ijk}}{\sin^2\theta_{ijk}^0}{\left[\cos\theta-\cos\theta_{ijk}^0\right]^2}$

- Four-body interaction
  
  Torsional bending (dihedral angle): $E(\Phi)=k_{ijkl}\left[1-\cos\left(n_{jk}(\Phi-\Phi_0)\right)\right]$
  
  Where $k_{ijkl}$ is the torsion barrier and $n_{jk}$ is the number of times the minimum value occurs.

$$
\begin{aligned}
\text{U}& =\sum_\text{bonds }{ \frac 1 2 }k_r(r_{ij}-r_0)^2+\sum_\text{angles }{ \frac 1 2 }k_\theta(\theta_{ijk}-\theta_0)^2  \\
&+\sum_\text{torsions }{ \sum _ n }k_{\phi,n}[\cos(n\phi_{ijkl}+\delta_n)+1] \\
&+\sum_\text{non-bonded pairs}{ \left [ \frac { q _ i q _ j }{ 4 \pi \varepsilon _ 0 r _ { i j }}+\frac{A_{ij}}{r_{ij}^{12}}-\frac{B_{ij}}{r_{ij}^6}\right]}
\end{aligned}
$$

Here, the first and second terms describe the bond vibration and bending described by the harmonic potential, the third term describes the bond torsional bending, and the fourth term describes the Coulomb and van der Waals interactions.

When calculating van der Waals and Coulomb interactions, we exclude interactions between certain pairs of atoms:

- Bonded atoms

- Atoms separated by two covalent bonds (1-3)

- Atoms separated by three covalent bonds (1-4)

#### 4.3.1 Features

- Element type alone is insufficient to determine interatomic interactions.

  For example, $sp^3$ and $sp^2$ carbon behave differently.

- In traditional molecular force fields, the user specifies the force field type to establish atomic properties.

  - The **force field type** and **atomic connectivity** must be determined before an MD simulation.

  - The force field type and interatomic connectivity remain fixed during the run.

- Reactive force fields (such as ReaxFF and REBO) describe interactions purely based on atomic positions and **allow atoms to change their coordination and environment**.

#### 4.3.2 Stillinger-Weber potential

$$
\begin{aligned}U=\sum_i\sum_{j\neq i}f_2(r_{ij})+\sum_i\sum_{j\neq i}\sum_{k\neq i,k\neq j}f_3(r_{ij},r_{ik},\cos\theta_{ijk})\end{aligned}
$$

It is widely used in semiconductor materials such as $GaN$ and $CdTe$, but cannot describe the non-tetrahedral structure of Si.

#### 4.3.3 Tersoff potential

$$
E_b=\sum_i\sum_{j>i}[a_{ij}V^R(r_{ij})-b_{ij}V^A(r_{ij})]
$$

Where $b_{ij}$ is the [Bond Order](https://en.wikipedia.org/wiki/Bond_order). The potential energy of $Si$ depends largely on the bond order.

$$
\begin{gathered}
b_{ij}=\left(1+\beta^{n}\zeta_{ij}^{n}\right)^{-1/2n}\\
\zeta_{ij}=\sum_{k\neq i,j}f_{C}(r_{ik})g\bigl(\theta_{ijk}\bigr)\exp\left[\lambda_{3}^{3}\bigl(r_{ij}-r_{ik}\bigr)^{3}\right] \\
g(\theta)=1+c^{2}/d^{2}-c^{2}/\left[d^{2}+(h-\cos\theta)^{2}\right] 
\end{gathered}
$$

By introducing a BO parameter that contains local structural information, the attractive potential is modified.

The Tersoff potential can accurately simulate the **metastable structures** of various covalent systems and was quickly extended to systems such as $SiC$ and $SiO_2$.

> **Bond order**:
> 
> The bond order determines the local bond energy between two atoms.
> 
> - Single bond, double bond, triple bond
> 
> - Atomic local environment$\Rightarrow$Bond order$\Rightarrow$Covalent energy

The introduction of bond order makes the potential dependent on the atomic environment.

Bond stretching, angles, and torsion terms depend on the bond order involved. Bond order is a pre-factor for all action terms. As bond order approaches zero, all involved bond energy terms also approach zero. Bond order also penalizes over- and under-bonding, thereby limiting the number of bonds an atom can form.

References：

1. [*Phys. Rev. B* **37**, 6991 (1988)](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.37.6991)
2. [*Phys. Rev. B* **42**, 9458 (1990)](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.42.9458)
3. [*J. Phys. Chem. A* **105**, 9396 (2001)](https://pubs.acs.org/doi/10.1021/jp004368u)



#### 4.3.4 REBO and AIREBO

The Tersoff potential accounts for different hybridization forms of carbon but cannot handle the nonlocal conjugated bonds of graphite. Brenner improved the analytical form of $b_{ij}$, accounting for $C-H$ interactions, improving the description of free radicals, and developing the Reactive Empirical Bond Order (REBO). To describe complex $CH$ macromolecules, Stuart et al. introduced **van der Waals forces** and **four-body torsion** on the basis of the original REBO potential function, developing the Adaptive Intermolecular Reactive Empirical Bond Order (AIREBO).

$$
\begin{gathered}
E=\sum_{i}E_{i}={\frac{1}{2}}\sum_{i\neq j}V_{ij} \\
V_{ij}=f_{C}(r_{ij})[f_{R}(r_{ij})+b_{ij}f_{A}(r_{ij})]\\
{{b_{ij}=\left(1+\beta^{n}\zeta_{ij}^{n}\right)^{-\frac{1}{2n}}}}\\{\downarrow} \\
E_{b} =\sum_{i}\sum_{j\neq i}[V_{R}(r_{ij})-\bar{B}_{ij}V_{A}(r_{ij})]\\{\downarrow}\\
E=\frac{1}{2}\sum_{i}\sum_{j\ne i}\left[E_{ij}^\text{REBO}{ + }E_{ij}^\text{LJ}{ + }\sum_{k\ne i,j}\sum_{l\ne i,j,k}E_{kijl}^\text{tors}\right]
\end{gathered}
$$

AIREBO solves the simulation problem of non-local conjugated bonds in graphite. The REBO family is widely used in the simulation of **carbon nanomaterials**.



#### 4.3.5 ReaxFF potential



#### 4.3.6 Machine Learning Interatomic Potential (MLIP)

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/5a723ba94f304eff9bb7354c991fc873/c4c939bd-5351-4924-bc05-5b078d0b01e4.png)

- Semi-empirical potentials:

  - Assume a given functional form

  - Determine parameters using theory and simulation

  - Access to some experimental data

    - Molecular bond lengths, binding energies

    - Gas-phase atomic scattering

    - Virial coefficients, gas-phase transport

    - Solid cohesive energy, lattice constants, elastic modulus

    - Melting point, critical point, triple point, surface tension

    - Point defects, surface energy, diffusion barriers, etc.

- Machine learning potentials:

  - Replace analytical functional forms with machine learning models

  - Determine parameters using first-principles data

  - Easily extendible to other bonded systems

MLIP is accurate and reliable, and significantly more efficient than ab initio calculations. This makes atomistic simulations on large temporal and spatial scales possible.

# Optimization Algorithm

---


## 1 Introduction to Optimization Algorithms

Optimization is a crucial problem in science, engineering, and finance. The goal of an optimization algorithm is to find the minimum or maximum value of a function.

In computational materials science, the potential energy surface of a molecule is often a complex, multidimensional function that depends on coordinates. Optimization algorithms are required to calculate the optimal molecular configuration.

Examples:

- [Travelling salesman problem (TSP)](https://en.wikipedia.org/wiki/Travelling_salesman_problem)

- Circuit Board Design

- First-principles calculations and molecular dynamics simulations

Concepts from simulations give rise to new optimization algorithms:

- [Gradient descent](https://en.wikipedia.org/wiki/Gradient_descent)

- [Simulated annealing](https://en.wikipedia.org/wiki/Simulated_annealing)

- [Genetic algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm)

### 1.1 Local optimization and global optimization

- Local Optimization Algorithms

  - Direct Methods (Using **the Objective Function Value**)

    - Coordinate Rotation Methods

    - Powell Method

  - Indirect Methods (Using **the First or Second Derivatives of the Objective Function**)
    
    - [Steepest descent method](https://en.wikipedia.org/wiki/Method_of_steepest_descent)
    
    - [Conjugate gradient method](https://en.wikipedia.org/wiki/Conjugate_gradient_method)
    
    - [Newton's method](https://en.wikipedia.org/wiki/Newton%27s_method)
  
  The **indirect method** has a faster convergence rate, and the **direct method** has higher reliability.

- Global Optimization Algorithm

  - Grid Search
  
  - [Genetic algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm)
  
  - [Simulated annealing](https://en.wikipedia.org/wiki/Simulated_annealing)

The local optimal solution of a **convex function** is the global optimal solution.

### 1.2 Optimize algorithm process

Given an initial point $x^0$,

1. At the iteration point $x^k$, the search direction $p^k$ is determined according to a certain rule;

2. Determine the appropriate search step size $\alpha_k$ along the search direction;

3. Let $x^{k+1}=x^k+\alpha_kp^k$,
   
   1. If $x^{k+1}$ satisfies the termination condition, the iteration stops and the approximate optimal solution $x^*$ is obtained;
   
   2. Otherwise, repeat the above steps.

Three elements of the optimization algorithm:

- Initial point $x^0$

- Search direction $p^k$

- Iteration step size $\alpha_k$

Among them, **search direction** is the most prominent and important. Determining the search direction is one of the most fundamental tasks in studying optimization algorithms.

## 2 Steepest descent

### 2.1 Core idea

- [Greedy algorithm](https://en.wikipedia.org/wiki/Greedy_algorithm) - Never go uphill;

- The optimization will go to the nearest local minimum.

### 2.2 Quadratic Steepest Descent Method

The **negative gradient** is the direction in which the function value decreases fastest in the range around that point.

$$
-\nabla f(X)=-\begin{pmatrix}\frac{\partial f}{\partial x}\\\\\frac{\partial f}{\partial y}\end{pmatrix}
$$

A quadratic form is a quadratic homogeneous polynomial in $n$ variables, and has a **symmetric matrix** representation:

$$
f(x)=\frac12x^TAx-b^Tx+c
$$

Its gradient is expressed as

$$
\nabla f(x)=Ax{-}b
$$

Then the search direction (negative gradient) is

$$
p=b{-}Ax
$$

Note: The quadratic form used to study the method of steepest descent must be **positive definite** to ensure that it has a minimum.

**The optimal step size for a positive definite quadratic form can be obtained analytically**:

$$
\alpha_k=\operatorname*{argmin}_\alpha\{\varphi(\alpha)=f(x^k+\alpha p^k)\}
$$

According to the definition of directional derivative $\nabla_pf=\frac{\partial f}{\partial x}p_x+\frac{\partial f}{\partial y}p_y+\frac{\partial f}{\partial z}p_z$, the derivative of the quadratic form $f(x)$ at the point $(x^k+\alpha p^k)$ along the negative gradient direction $p^k$ is

$$
\varphi^{\prime}(\alpha)=(x^k+\alpha p^k)^TAp^k-b^Tp^k
$$

Let $\varphi^{\prime}(\alpha)=0$, that is: $\alpha(p^k)^TAp^k=-((x^k)^TA-b^T)p^k$

Since $(x^k)^TA-b^T=(-p^k)^T$，we get$\alpha=\frac{(p^k,p^k)}{(Ap^k,p^k)}$

Take $\color{red}\alpha_k=\frac{(p^k,p^k)}{(Ap^k,p^k)}$，then $\alpha_k$ is the **optimal step length** of $\boldsymbol{k}\to\boldsymbol{k}+\boldsymbol{1}$.

### 2.3 Code Implementation

$$
\alpha_k=\frac{(p^k,p^k)}{(Ap^k,p^k)}\quad\quad\quad\quad x^{k+1}=x^k+\alpha^kp^k
$$

```python
def steepest_descent(x0, tol=1e-5, max_iter=1000):
    x = x0
    for i in range(max_iter):
        p = -grad_f(x)
        if np.linalg.norm(p) < tol:
            break
        alpha = p.T @ p / (p.T @ A @ p)
        x = x + alpha * p
    return x
```

### 2.4 Disadvantages of steepest descent

- As the iterations continue, the search direction exhibits **oscillatory** behavior.

- The function gradients at two adjacent iteration points are perpendicular to each other, indicating that the two adjacent search directions (negative gradients) are perpendicular to each other.
  
  $$
  \begin{aligned}x^{k+1}&=x^k+\alpha p^k\\\frac d{d\alpha}f(x^{k+1})&=f^{\prime}(x^{k+1})^T\frac d{d\alpha}x^{k+1}=f^{\prime}(x^{k+1})^T•p^k\equiv0\end{aligned}
  $$
  
  Where $p^k$ is the old gradient direction and $f^{\prime}(x^{k+1})$ is the new gradient direction.

## 3 One-dimensional search algorithm

### 3.1 Steepest descent of general function forms

For general function forms, the optimal step size can be determined by **numerical solution**.

$$
\alpha_k=\operatorname*{argmin}_\alpha\{\varphi(\alpha)=f(x^k+\alpha p^k)\}
$$

- Exact One-Dimensional Search

  - Back-and-Forward Method

  - Segmentation Method

  - Interpolation Method

- Inexact One-Dimensional Search (Backtracking-Armijo)

### 3.2 Determining the step size of one-dimensional search

1. **Determine the search interval** so that the function $f(x)$ has a local minimum
   
   - Graphical methods
   
   - Analytical methods
   
   - Bracketing Method

> Determining the search range by **bracketing method**:
> 
> Compare the sizes of $f(x_0)$ and $f(x_0+\Delta x)$,
> 
> - The direction is correct, **forward operation**, increase the step size
> 
> - Opposite direction, **backward operation**, reverse search
> 
> Once the function value satisfies the high-low-high relationship, the search interval can be determined.

2. **Narrow the search interval** to determine the exact location of the minimum value
   
   - Golden Section
   
   - dichotomy
   
   - Interpolation
   
   - Newton's method

> Narrowing the search range by **two-point cubic interpolation**:
> 
> In the search interval, the objective function is continuously approximated by a cubic polynomial, and the minimum point of the interpolation polynomial is gradually used to approximate the minimum point of the one-dimensional search problem.
> 
> Known conditions: $f(x_1),f(x_2),f^{\prime}(x_1),f^{\prime}(x_2)$
> 
> Construct function$\varphi(x)=a(x-x_1)^3+b(x-x_1)^2+c(x-x_1)+f(x_1)$
> 
> Solving for coefficients based on boundary conditions.
> 
> $$
\begin{gathered}
a=\frac{f^{\prime}(x_1)+f^{\prime}(x_2)}{(x_2-x_1)^2}-2\frac{f(x_2)-f(x_1)}{(x_2-x_1)^3} \\
b=-\frac{2f^{\prime}(x_1)+f^{\prime}(x_2)}{(x_2-x_1)}+\frac{3(f(x_2)-f(x_1))}{(x_2-x_1)^2} \\
c=f^{\prime}(x_1) 
\end{gathered}
$$
> 
> Get the extreme points of the cubic polynomial
> 
> $$
x_{new}=x_1+\frac{-c}{b+\sqrt{b^2-3ac}}
$$

### 3.3 Inexact Search - [Backtracking Line Search (Armijo Criterion)](https://en.wikipedia.org/wiki/Backtracking_line_search)

Exactly solving each one-dimensional subproblem often requires a large amount of computation. A more practical approach is to use an inexact line search to obtain the step size that sufficiently reduces the objective function at the lowest cost.

**Armijo Criterion**：

$$
\begin{aligned}&f(x^k+\alpha_kp_k)-f(x^k)\leq\alpha_k\beta g_k^Tp_k\end{aligned}
$$

> [**Rosenbrock function**](https://en.wikipedia.org/wiki/Rosenbrock_function)：
> 
> $$
f(x,y)=(1-x)^2+100(y-x^2)^2
$$
> 
> - Non-convex function. In the field of optimization, it is often used as a performance test function of an optimization algorithm;
> 
> - The global optimum is located in a narrow, flat, parabolic "valley".

## 4 Conjugate Gradient

### 4.1 Conjugate direction

To avoid oscillation, the next iterative search direction is directed directly to the minimum point.

$$
\begin{gathered}
x^1=x^0+\alpha_0d^0 \\
\left.\frac{\partial f}{\partial\alpha}\right|_{x^1}=\nabla f(x^1)^Td^0=0 \\
x^*=x^1+\alpha_1d^1 
\end{gathered}
$$

The necessary condition for the quadratic function $f(x)$ to reach its minimum point at $x^*$ is

$$
\begin{aligned}\nabla f(x^*)&=Ax^*-b=0\\\\\nabla f(x^*)&=A(x^1+\alpha_1d^1)-b=Ax^1-b+\alpha_1Ad^1\\\\&=\nabla f(x^1)+\alpha_1Ad^1=0\end{aligned}
$$

Multiplying both sides of the equation by $(d^0)^T$ yields $\color{red}{(d^0)^TAd^1=0}$

$d^0$ and $d^1$ are conjugate directions with respect to $A$. If $A=I$, then $d^0$ and $d^1$ are orthogonal.

### 4.2 Conjugated direction construction method

A set of $n$ linearly independent vectors can be constructed through [Gram-Schmidt orthogonalization](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process) to construct the **conjugate direction** as follows:

$$
\begin{aligned}
&u_1=\boldsymbol{v}_1  \\
&u_{2}=v_{2}-\frac{v_{2}Au_{1}}{u_{1}Au_{1}}u_{1} \\
&u_{3}=v_{3}-\frac{v_{3}Au_{1}}{u_{1}Au_{1}}-\frac{v_{3}Au_{2}}{uAu_{2}}u_{2} \\
\end{aligned}
$$

The conjugate direction is constructed by the negative gradient of the iteration point, so it is called the conjugate gradient method.

$$
f(x)=\frac12x^TAx-b^Tx+c
$$

Starting from point $x^k$, perform a one-dimensional search along a conjugate direction $d^k$ of $A$ to reach $x^{k+1}$

$$
x^{k+1}=x^k+\alpha_kd^k
$$

The gradients at points $x^k$ and $x^{k+1}$ are

$$
g^k=Ax^k-b\quad g^{k+1}=Ax^{k+1}-b
$$

$d^0=-g^0=-Ax^0+b$, According to the properties of one-dimensional exact search, $(g^1)^Td^0=0$

let $d^1=-g^1+\beta_0d^0$, then $(d^1)^T=-(g^1)^T+\beta_0(d^0)^T$

Choose $\beta_0$ so that $(d^1)^TAd^0=0$, and multiply both sides of the above equation by $Ad^0$ to get

$$
\begin{aligned}
(d^1)^TAd^0& =-(g^1)^TAd^0+\beta_0(d^0)^TAd^0  \\
\boldsymbol{\beta}_{\mathbf{0}}& =\frac{(g^1)^TAd^0}{(d^0)^TAd^0} 
\end{aligned}
$$

### 4.3 Conjugate gradient algorithm process

1. One-dimensional search along the $d^{k-1}$ direction
   
   $$
   x^k=x^{k-1}+\alpha_{k-1}d^{k-1}\tag{1}
   $$

2. Construct new conjugation directions
   
   $$
\begin{aligned}d^k&=-g^k+\beta_{k-1}d^{k-1}\\\beta_{k-1}&=\frac{(g^k)^TAd^{k-1}}{(d^{k-1})^TAd^{k-1}}\rightarrow\beta_{k-1}=\frac{(g^k)^Tg^k}{(g^{k-1})^Tg^{k-1}}\end{aligned}
   $$
   
   The new conjugate direction only requires the current negative gradient and the direction information of the previous step.

3. One-dimensional search along the new conjugate direction $d^{k-1}$
   
   $$
   x^{k+1}=x^k+\alpha_kd^k\tag{3}
   $$

### 4.4 Code Implementation

```python
def linear_conjugate_gradient(A, b, x0, max_iter, tol=1e-5):
    # Initializations
    x = x0; r= b - A @ x ; p = r 
    rsold = r.T @ r
    for i in range(max_iter):
        Ap = A @ p
        alpha = rsold / (p.T @ Ap)
        x += alpha * p                # Equation.(3)
        r -= alpha * Ap
        rsnew = r.T @ r
        # Check for convergence
        if np.sqrt(rsnew) < tol:
            break
        p = r + (rsnew / rsold) * p    # Equation.(2)
        rsold = rsnew
    return x
```

## 5 Configuration optimization of multi-atom systems

### 5.1 Energy minimization

Molecular mechanics, also known as force field methods, is essentially an **energy minimization method**. This involves modifying the geometry of particles under the influence of **interatomic potentials**, using **energy minimization** as the criterion to determine the optimal configuration of a system.

Using the reduced L-J potential as an example, we construct a two-dimensional initial configuration containing five atoms and attempt to use an optimization algorithm to obtain the lowest-energy configuration.

$$
U(r_{ij})=4[(\frac1{r_{ij}})^{12}-(\frac1{r_{ij}})^6]
$$

### 5.2 Total energy of the system

In order to facilitate the application of the optimization algorithm, it is necessary to construct a functional relationship from atomic coordinates to total energy.

$$
\begin{aligned}
X&=(x_{1},y_{1},x_{2},y_{2},x_{3},y_{3},x_{4},y_{4},x_{5},y_{5})  \\
r_{ij}&=\sqrt{(y_{j}-y_{i})^{2}+(x_{j}-x_{i})^{2}} \\
E(X)&=\sum_{i=1}^{4}\sum_{j=i+1}^{5}U(r_{ij})=\sum_{i=1}^{4}\sum_{j=i+1}^{5}4[(\frac{1}{r_{ij}})^{12}-(\frac{1}{r_{ij}})^{6}] \\
&=\sum_{i=1}^4\sum_{j=i+1}^54[\frac1{((y_j-y_i)^2+(x_j-x_i)^2)^6}-\frac1{((y_j-y_i)^2+(x_j-x_i)^2)^3}]
\end{aligned}
$$

### 5.3 Code Implementation

```python
# Define Reduced LJ potential
def LJ_reduced_pot(r):
    return 4*((1.0/r)**12-(1.0/r)**6)

# Define derivative
def LJ_reduced_force(r):
    return 24*(2*(1/r)**12-(1/r)**6)/r

# Generate random configuration
def genPts(N):
    np.random.seed(10)
    pts = np.zeros(N*2)
    for i in range(N*2):
        pts[i] = np.random.uniform(-2,2)
    return pts

# Define the objective function
def E(x):
    N = int(len(x)/2)
    eng_pot = np.zeros(N)
    R = x.reshape(N,2)
    for i in range(N-1):
        for j in range(i+1,N):
            Rij = R[j]-R[i]
            r = np.linalg.norm(Rij)
            eng_pot[i] += 0.5*LJ_reduced_pot(r)
            eng_pot[j] += 0.5*LJ_reduced_pot(r)
    return np.sum(eng_pot)

# Define derivative
def dEdR(x):
    ......

# Geometry optimization
W1 = SD_search(E, dEdR, x)
```

# Integration algorithm

---

The core of molecular dynamics evolution is to solve the **Newton's equations of motion**.

For a molecular system consisting of $N$ atoms, let the coordinates, velocity, momentum, and force of the $i$th atom be $r_{i}(t)$, $\nu_{i}(t)$, $p_{i}(t)$, $f_{i}(r,t)$, respectively, and its initial values be $r_{i}(0)$, $\nu_{i}(0)$, $p_{i}(0)$ and $f_{i}(0)$. Then the Newton equations of motion for the $i$th atom are

$$
f_i=m_i\frac{d^2r_i}{dt^2}=m_i\ddot{r}_i\quad{(i=1,2,3,....N)}
$$

Under unconstrained conditions, there are $3N$ degrees of freedom, i.e., a second-order differential equation with $3N$ components.

By solving the $3N$ Newton equations of motion based on the system's initial state, we can obtain the motion trajectories of all atoms in the system using an integration algorithm.

The motion trajectory of the $i$th particle

$$
r_i(t)=\int v_i(t)dt
$$

When the analytical form of $V_i(t)$ is unknown, the computer will **discretize** the continuous integral to find an approximate solution

$$
r_i(t)=\sum_0^tv_i(t)\Delta t
$$

Molecular dynamics evolution trajectories lie above the potential energy surface (kinetic energy contribution), requiring a good distribution of points in phase space.

Molecular dynamics integration algorithm requirements

- Fast speed and low memory usage

- Can use long time steps $\Delta t$

- Can approximate and replicate classical paths

- Satisfy energy and momentum conservation and time reversibility

- Simple form and easy programming

## 1 [Forward Euler Method](https://en.wikipedia.org/wiki/Euler_method)

One-dimensional harmonic oscillator equation of motion

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/b0a9b5b0e78e4249853bcd60a2114784/340609a9-b8b9-4179-a40a-2ce23a6d8006.png)

$$
\begin{aligned}\frac{md^2x}{dt^2}&=f=-kx\quad\quad x(0)=0,v(0)=v_0\\\\x(t)&=\frac{v_0}{\omega}\sin(\omega t)\quad\quad \omega=\sqrt{\frac km}\\\\E(t)&=K(t)+V(t)=\frac{1}{2}mv_0^2=E(t=0)=\color{red}{const.}\end{aligned}
$$

### 1.1 Core idea

Numerical integration is used to calculate the motion trajectories $r(t)$ and $v(t)$ in phase space, transforming the **differential equations** into **finite difference equations** for numerical computation.

First, the time step for the differential calculation is set to $\Delta t$. The forward difference quotient representation of the first-order differential form of the finite difference method is used, directly applying **the first-order Taylor expansion formula** expanded to $\Delta t$:

$$
f(t+\Delta t)=f(t)+\Delta t\frac{df}{dt}+O(\Delta t^2)
$$

After Taylor expansion of $r(t)$ and $v(t$, we get the **forward Euler algorithm**:

$$
\begin{aligned}r(t+\Delta t)&=r(t)+v(t)\Delta t+O(\Delta t^2)\\\\v(t+\Delta t)&=v(t)+\frac{F(t)}m\Delta t+O(\Delta t^2)\end{aligned}
$$

### 1.2 Code Implementation

```python
class HarmonicOscillator:
    def __init__(self, k, m):
        self.k = k
        self.m = m
    def acceleration(self, x):
        return -self.k * x / self.m


class ForwardEulerIntegrator:
    ......
    def step(self, system, x, v):
        """
        Perform one integration step.

        x :Current position.
        v : Current velocity.
        """
        a = system.acceleration(x)
        x_new = x + self.dt * v
        v_new = v + self.dt * a
        return x_new, v_new
```

### 1.3 Algorithm flaws

#### 1.3.1 Actual solution results

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/b0a9b5b0e78e4249853bcd60a2114784/18203e1f-7257-401e-9e77-6eee9e5a69bf.png)

When using the forward Euler algorithm to calculate the energy of a system in molecular dynamics, **overheating** often occurs, that is, **the calculated energy is greater than the actual energy of the system**, and may even cause **the system energy to not converge**.

#### 1.3.2 Physical image of system overheating

$$
\begin{aligned}&\text{Actual location:}\quad&r^{\prime}(t+\Delta t)&=r(t)+\bar{v}\Delta t\\\\&\text{Predicted location:}\quad&r(t+\Delta t)&=r(t)+v(t)\Delta t\end{aligned}
$$

- ${\nu(t)\geq\bar{\nu}}$, the predicted position is greater than the actual position - **potential energy is too large**

$$
\begin{aligned}\text{Actual speed:}\quad&\quad\nu^{\prime}(t+\Delta t)=v(t)+\frac{\bar{F}}{m}\Delta t\\\\\text{Predicted location:}\quad&\quad\nu(t+\Delta t)=\nu(t)+\frac{F(t)}{m}\Delta t\end{aligned}
$$

- $|F(t)|\leq|\bar{F}|$, the predicted speed is greater than the actual speed——**Kinetic energy is too large**

#### 1.3.3 Theoretical proof of system overheating

$$
\begin{aligned}r(t+\Delta t)&=r(t)+v(t)\Delta t+O(\Delta\textbf{t}^2)\\\\v(t+\Delta t)&=v(t)+\frac{-dV(t)}{\text{m}dr}\Delta t+O(\Delta t^2)\end{aligned}
$$

Ignore the second-order small quantity, substitute into the total energy expression of the system, and calculate the energy difference between two adjacent steps

$$
\begin{gathered}
E(t+\Delta t)-E(t)=\frac{1}{2}m\left(v(t)+\frac{-dV(t)}{mdr}\Delta t\right)^{2}+V(t)+\frac{dV(t)}{dt}\Delta t-\frac{1}{2}mv(t)^{2}-V(t) \\
=-v(t)\frac{dV(t)}{dr}\Delta t+\frac{dV(t)}{dt}\Delta t+\frac{1}{2m}\biggl[\frac{dV(t)}{dr}\Delta t\biggr]^{2} \\
=-\frac{dr}{dt}\frac{dV(t)}{dr}\Delta t+\frac{dV(t)}{dt}\Delta t+\frac{1}{2m}\biggl[\frac{dV(t)}{dr}\Delta t\biggr]^{2} \\
=\frac{1}{2m}\biggl[\frac{dV(t)}{dr}\Delta t\biggr]^{2} \color{red}> 0
\end{gathered}
$$

**Total energy increases, the system overheats, and the forward Euler algorithm cannot guarantee energy conservation! **

## 2 [Verlet Integration](https://en.wikipedia.org/wiki/Verlet_integration)

### 2.1 Position Verlet

$$
\begin{aligned}r(t+\Delta t)&=r(t)+\dot{r}(t)\Delta t+\frac{\ddot{r}}2\Delta t^2+\frac16\ddot{r}\Delta t^3+\mathcal{O}(\Delta t^4)\\r(t-\Delta t)&=r(t)-\dot{r}(t)\Delta t+\frac{\ddot{r}}2\Delta t^2-\frac16\ddot{r}\Delta t^3+\mathcal{O}(\Delta t^4)\end{aligned}
$$

- **Location Update**
  
  The sum of the above formula is:

$$
r(t+\Delta t)=2r(t)-r(t-\Delta t)+\frac{F(t)}m\Delta t^2+O(\Delta t^4)
$$

- **Speed Update**
  
  The difference of the above formula is:

$$
v(t)=\frac{r(t+\Delta t)-r(t-\Delta t)}{2\Delta t}+O(\Delta t^2)
$$

In the Verlet algorithm, **position update does not require velocity information**, but velocity must be calculated to obtain kinetic energy information.

#### 2.1.1 Startup of the Verlet algorithm

$$
r_{n+1}=2r_n-r_{n-1}+\left(\frac{F_n}m\right)\Delta t^2+O(\Delta t^4)
$$

When the algorithm starts, $n=0$, and $r_0$ is known, the value of $r_{-1}$ must be known before the calculation of $r_1$ can begin. To avoid calculating $r_{-1}$, the Taylor expansion is usually used to update the first step:

$$
r(t+\Delta t)=r(t)+\nu(t)\Delta t+\frac{a(t)}2\Delta t^2
$$

Then, the calculated $r_0$ and $r_1$ are used to continuously update the position according to the Verlet algorithm.

#### 2.1.2 Verlet algorithm diagram

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/b0a9b5b0e78e4249853bcd60a2114784/dd965f23-9fd5-489c-a170-918fe46715e1.png)

#### 2.1.3 Verlet algorithm code implementation

```python
def step(self, system, x, v):
    if self.previous_x is None:
        self.previous_x =x-v0*self.dt+0.5*system.acceleration(x)* self.dt ** 2
    # Calculate new position using Verlet algorithm
    a = system.acceleration(x)
    new_x = 2*x - self.previous_x + a * self.dt ** 2
    # Calculate the velocity for the current position
    if self.previous_x is not None:
        current_v = (new_x - self.previous_x) / (2*self.dt)
    else:
        current_v = v0
    # Update previous_x for the next step
    self.previous_x = x
    return new_x, current_v
```

#### 2.1.4 Verlet algorithm features

- Advantages

  - High position accuracy, with an error of $O(\Delta t^4)$

  - Force is calculated only once per integration

  - Time reversible

- Disadvantages

  - Large velocity error

  - Trajectory is independent of velocity and cannot be coupled to a heat bath

### 2.2 Velocity Verlet

Perform **second-order Taylor expansion** on $r(t)$ and $v(t$:

$$
\begin{aligned}&r(t+\Delta t)=r(t)+\nu(t)\Delta t+\frac12a(t)\Delta t^2+O(\Delta t^3)\\\\&v(t+\Delta t)=v(t)+a(t)\Delta t+\frac12\dot{a}(t)\Delta t^2+O(\Delta t^3)\end{aligned}
$$

where $\dot{a}(t)=\frac{a(t+\Delta t)-a(t)}{\Delta t}$, substituting into the velocity equation gives

$$
v(t+\Delta t)=v(t)+\frac12[a(t)+a(t+\Delta t)]\Delta t+O(\Delta t^3)
$$

- **Location update**
  
  $$
  r(t+\Delta t)=r(t)+\nu(t)\Delta t+\frac12a(t)\Delta t^2
  $$

- **Speed update**
  
  $$
  v(t+\Delta t)=v(t)+\frac12[a(t)+a(t+\Delta t)]\Delta t
  $$

#### 2.2.1 Velocity Verlet algorithm diagram

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/b0a9b5b0e78e4249853bcd60a2114784/1baedf86-e1af-47be-be37-c4878432db7b.png)

#### 2.2.2 Half-step speed

- **Location update**
  
  $$
  r(t+\Delta t)=r(t)+\nu(t)\Delta t+\frac12a(t)\Delta t^2 \\
\boldsymbol{\rightarrow}\begin{cases}v\left(t+\frac{\Delta t}{2}\right)=v(t)+\frac{\Delta t}{2}a(t)\\{}\\r(t+\Delta t)=r(t)+v\left(t+\frac{\Delta t}{2}\right)\Delta t\end{cases}
  $$

- **Speed update**
  
  $$
  v(t+\Delta t)=v(t)+\frac12[a(t)+a(t+\Delta t)]\Delta t \\
\rightarrow\begin{cases}a(t+\Delta t)=\dfrac{F(r(t+\Delta t))}{m}\\[2ex]v(t+\Delta t)=v\left(t+\dfrac{\Delta t}{2}\right)+\dfrac{\Delta t}{2}a(t+\Delta t)\end{cases}
  $$

#### 2.2.3 Half-step speed diagram

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/b0a9b5b0e78e4249853bcd60a2114784/547986ee-917f-4116-b94f-9244f3f8fd2b.png)

#### 2.2.4 Velocity Verlet algorithm code implementation

```python
def step(self, system, x, v):
    a = system.acceleration(x)
    x_new = x + self.dt * v + 0.5 * self.dt**2 * a
    a_new = system.acceleration(x_new)
    v_new = v + 0.5 * self.dt * (a + a_new)
    return x_new, v_new
```

## 3 [Leapfrog Integration](https://en.wikipedia.org/wiki/Leapfrog_integration)

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/b0a9b5b0e78e4249853bcd60a2114784/54d9f56b-ce28-4915-815c-ab7e793cb157.png)

$$
\begin{aligned}r(t+\Delta t)&=r(t)+\Delta t\cdot v(t+\frac{\Delta t}{2})\\\\v(t+\frac{\Delta t}{2})&=v(t-\frac{\Delta t}{2})+\frac{\Delta t}{m}\cdot F(t)\end{aligned}
$$

### 3.1 Startup of the leapfrog algorithm

Initial step:

$$
\begin{aligned}r(\Delta t)&=r(0)+\Delta t\cdot v(\frac{\Delta t}2)\\\\v(\frac{\Delta t}2)&={\color{red}\nu(-\frac{\Delta t}2)}+\frac{\Delta t}m\cdot F(0)\end{aligned}
$$

where

$$
v\left(-\frac{\Delta t}2\right)=v(0)-\frac{F(0)}{2m}\Delta t
$$

### 3.2 Leapfrog algorithm diagram

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/b0a9b5b0e78e4249853bcd60a2114784/8a49f65a-aae6-4da6-b9c4-dda171d81c3f.png)

### 3.3 Leapfrog algorithm features

- Advantages

  - Memory savings

  - Higher accuracy and stability

  - Improved accuracy of velocity calculations

- Disadvantages

  **Velocity and position always differ by half a time step and are not defined at the same time.** Therefore, kinetic and potential energy cannot be defined simultaneously, making it **impossible to directly calculate total energy**.

## 4 Stability of integration algorithm

### 4.1 Integration step

Particle evolution involves **the integration of position over time**, so the numerical calculation must consider the choice of $\Delta t$.

The selection of $\Delta t$ is a **compromise between accuracy and speed**.

- If $\Delta t$ is too small, it will not be possible to traverse the entire phase space within a finite time.

- If $\Delta t$ is too large, the integration algorithm will become unstable.

The $\Delta t$ commonly used in molecular dynamics is generally between $0.1\sim10fs$ ($10^{-15}s$).

- For **solids** or **molecular vibrations**, the time step is usually no longer than $\frac1{10}$ of the shortest period of motion.

- For **liquids**, the time step needs to be much smaller than the average collision time between two liquid molecules.

> The choice of step size $\Delta t$ is a trade-off between accuracy and speed.
>
> Simple principle: $\Delta t$ must be chosen to ensure **energy conservation** and **time reversibility** of the trajectory.
>
> Complex requirement: When the system contains multiple time scales, such as mixed particles of varying masses, solvent aggregates, or coexisting flexible and rigid molecules, **the step size must be chosen based on the component or model with the fastest dynamics**.

### 4.2 Algorithmic reversibility

#### 4.2.1 Newton's equations of motion

$$
F=m\frac{d^2r}{dt^2}
$$

Perform time reversal, i.e. $\color{red}t^{\prime}=-t$

$$
\begin{aligned}\frac{dr}{dt^{\prime}}&=-\frac{dr}{dt}\\\\F&=m\frac{d^2r}{dt^{\prime2}}\end{aligned}
$$

Newton's equations of motion remain unchanged and are time reversible.

#### 4.2.2 Forward Euler algorithm

$$
\begin{aligned}r(t+\Delta t)&=r(t)+v(t)\Delta t\\\\v(t+\Delta t)&=v(t)+\frac{F(t)}m\Delta t\end{aligned}
$$

Perform time reversal, i.e. $\color{red}t^{\prime}=-t$

$$
\begin{aligned}
r^\prime(t)& =r(t+\Delta t)-\nu(t+\Delta t)\Delta t  \\
&=r(t)-\frac{F(t)}m\Delta t^2 \\
&\neq r(t)
\end{aligned}
$$

$$
\begin{aligned}v'(t)&=v(t+\Delta t)-\frac{F(t+\Delta t)}m\Delta t\\&=v(t)-\frac{F'(t)}m\Delta t^2\\&\neq v(t)\end{aligned}
$$

**Forward Euler algorithm does not satisfy time reversibility.**

#### 4.2.3 Verlet algorithm

$$
r(t+\Delta t)=2r(t)-r(t-\Delta t)+\frac{F(t)}m\Delta t^2
$$

Perform time reversal, i.e. $\color{red}t^{\prime}=-t$

$$
\begin{aligned}
r^{\prime}(t-\Delta t)& =2r(t)-r(t+\Delta t)+\frac{F(t)}m\Delta t^{2}  \\
&=r(t-\Delta t)
\end{aligned}
$$

**Verlet algorithm satisfies time reversibility.**

#### 4.2.4 Velocity Verlet algorithm

$$
\begin{aligned}r(t+\Delta t)&=r(t)+v(t)\Delta t+\frac12a(t)\Delta t^2\\v(t+\Delta t)&=v(t)+\frac12[a(t)+a(t+\Delta t)]\Delta t\end{aligned}
$$

Perform time reversal, i.e. $\color{red}t^{\prime}=-t$

$$
\begin{aligned}r^{\prime}(t)&=r(t+\Delta t)-\nu(t+\Delta t)\Delta t+\frac12a(t+\Delta t)\Delta t^2 \\
&=r(t)\end{aligned}
$$

**The Velocity Verlet algorithm satisfies time reversibility.**

#### 4.2.5 Leapfrog algorithm

$$
\begin{gathered}
r(t+\Delta t)=r(t)+v(t+\frac{\Delta t}2)\Delta t \\
\begin{aligned}v(t+\frac{\Delta t}{2})&=v(t-\frac{\Delta t}{2})+\frac{F(t)}{m}\Delta t\end{aligned} 
\end{gathered}
$$

Perform time reversal, i.e. $\color{red}t^{\prime}=-t$

$$
\begin{aligned}r^{\prime}(t)&=r(t+\Delta t)-v\left(t+\frac{\Delta t}2\right)\Delta t=r(t)\\\\v^{\prime}(t-\frac{\Delta t}2)&=v\left(t+\frac{\Delta t}2\right)-\frac{F(t)}m\Delta t=v(t-\frac{\Delta t}2)\end{aligned}
$$

**The Leapfrog algorithm satisfies time reversibility.**

### 4.3 [Symplectic Structure](https://en.wikipedia.org/wiki/Symplectic_geometry)

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/b0a9b5b0e78e4249853bcd60a2114784/7edcef0d-9376-475f-8818-31760cee702f.png)

Before and after the evolution, $dA=dA^\prime$, that is, the area remains unchanged.

According to the [Liouville–Arnold theorem](https://en.wikipedia.org/wiki/Liouville%E2%80%93Arnold_theorem), a system evolving according to Newtonian mechanics follows a **symplectic structure**. This symplectic structure helps maintain the stability of the algorithm.

#### Mathematical Expression

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/16392/b0a9b5b0e78e4249853bcd60a2114784/3aebaa6a-f1a5-44ec-aadd-5133d2b818b6.png)

Area at time $t$: $dA=dxdp$

Area at $t^\prime$: $dA^{\prime}=|de_1\times de_2|$

Transformation formula:

$$
\begin{aligned}x^{\prime}&=X(x,p)\\\\p^{\prime}&=P(x,p)\end{aligned}
$$

$$
\begin{aligned}de_1&=\left(\frac{\partial X}{\partial x}\hat{x}+\frac{\partial P}{\partial x}\hat{p}\right)dx\\\\de_2&=\left(\frac{\partial X}{\partial p}\hat{x}+\frac{\partial P}{\partial p}\hat{p}\right)dp\end{aligned}
$$

Use ***Jacobian*** to determine whether it satisfies the symplectic structure:

$$
J=\begin{pmatrix}\frac{\partial X}{\partial x}&\frac{\partial X}{\partial p}\\\\\frac{\partial P}{\partial x}&\frac{\partial P}{\partial p}\end{pmatrix}
$$

- [ ] Forward Euler algorithm

- [x] Verlet algorithm

- [x] Velocity Verlet algorithm

- [x] Leapfrog algorithm